# Korean specific copper mixes

Kernel: `bw25`

This notebook assumes you have run notebooks 2 and 3 already.

In [1]:
import pandas as pd
import bw2data as bd
import bw2io as bi
import bw2analyzer as ba
import bw2calc as bc
import bw_processing as bwp
from pathlib import Path
import numpy as np
import csv
import sys

In [2]:
bd.projects.set_current('ei38-teaching-25')

Let's review our potential sources of copper, and their market fractions:

In [4]:
copper = bd.get_activity(database="ei 3.8 cutoff", name='market for copper concentrate, sulfide ore')
copper

'market for copper concentrate, sulfide ore' (kilogram, GLO, None)

In [5]:
for exc in copper.technosphere():
    print(exc)

Exchange: 0.0260899569876454 kilogram 'zinc mine operation' (kilogram, GLO, None) to 'market for copper concentrate, sulfide ore' (kilogram, GLO, None)>
Exchange: 0.0515377913932391 kilogram 'copper mine operation and beneficiation, sulfide ore' (kilogram, AU, None) to 'market for copper concentrate, sulfide ore' (kilogram, GLO, None)>
Exchange: 0.0385427501095062 kilogram 'copper mine operation and beneficiation, sulfide ore' (kilogram, CA, None) to 'market for copper concentrate, sulfide ore' (kilogram, GLO, None)>
Exchange: 0.220395946156588 kilogram 'copper mine operation and beneficiation, sulfide ore' (kilogram, CL, None) to 'market for copper concentrate, sulfide ore' (kilogram, GLO, None)>
Exchange: 0.092347757599498 kilogram 'copper mine operation and beneficiation, sulfide ore' (kilogram, CN, None) to 'market for copper concentrate, sulfide ore' (kilogram, GLO, None)>
Exchange: 0.0317023807993413 kilogram 'copper mine operation and beneficiation, sulfide ore' (kilogram, ID, N

Getting region-specific data on copper co-production will be difficult, as ecoinvent doesn't have much geographic details on these mines (e.g. molybdenite, zinc, gold, and nickel production). Let's see how much is produced in primarily copper mines:

In [6]:
sum([
    exc['amount']
    for exc in copper.technosphere()
    if exc.input['name'] == 'copper mine operation and beneficiation, sulfide ore'
])

0.8380748170283512

I think that this is good enough. Now, we know that we have cooper production in Exiobase:

In [9]:
cu_kr_exiobase = bd.get_activity(name='Copper production', database="EXIOBASE 3.8.1 2017 monetary", location='KR')

However, I am not sure that we should take this activity as the mix of copper being used in industry, and particularly in the car parts industry. Instead, let's take advantage of the fact that we have the whole Exiobase database to do an inventory calculation. For our functional unit, we can use `Manufacture of electrical machinery and apparatus n.e.c.`, and get the *respective* copper fractions from different Exiobase locations.

In [11]:
lca = bc.LCA(
    {bd.get_activity(
        name="Manufacture of electrical machinery and apparatus n.e.c.", 
        database="EXIOBASE 3.8.1 2017 monetary", 
        location='KR',
    ): 1}
)
lca.lci()

In [12]:
copper_inputs_kr_elec_machinery = sorted([
    (lca.supply_array[lca.dicts.product[node.id]], node)
    for node in bd.Database("EXIOBASE 3.8.1 2017 monetary")
    if node['name'] == 'Copper production'
], reverse=True)

In [19]:
copper_inputs_kr_elec_machinery[:15]

[(0.056738717215526764, 'Copper production' (million €, KR, None)),
 (0.009317991884388758, 'Copper production' (million €, WL, None)),
 (0.006975002935027911, 'Copper production' (million €, CN, None)),
 (0.003109505241170962, 'Copper production' (million €, WM, None)),
 (0.0029717316839159586, 'Copper production' (million €, JP, None)),
 (0.002742825955020737, 'Copper production' (million €, US, None)),
 (0.0017994500131776612, 'Copper production' (million €, WA, None)),
 (0.001438144712277453, 'Copper production' (million €, IN, None)),
 (0.0009033302983175338, 'Copper production' (million €, DE, None)),
 (0.000854596373371257, 'Copper production' (million €, AU, None)),
 (0.000837873645738335, 'Copper production' (million €, TW, None)),
 (0.000833554611307664, 'Copper production' (million €, ES, None)),
 (0.0006163921286590385, 'Copper production' (million €, WF, None)),
 (0.0005549428668933038, 'Copper production' (million €, MX, None)),
 (0.00048093280842529117, 'Copper productio

And here are the locations for copper production we are considering:

In [18]:
sorted([
    x['location'] 
    for x in bd.Database("ei 3.8 cutoff") 
    if x['name'] == 'copper mine operation and beneficiation, sulfide ore'
    and x['reference product'] == 'copper concentrate, sulfide ore'
])

['AU', 'CA', 'CL', 'CN', 'ID', 'KZ', 'RU', 'RoW', 'US', 'ZM']

Here are the definitions of Exiobase "RoW" regions:

* WA: Asia and Pacific. Excludes ID, JP, TW, IN, RU, CN, KR, AU
* WL: Americas. Excludes BR, US, CA, MX
* WE: Europe. 
* WF: Africa. Excludes ZA
* WM: Middle East. Excludes TR

We also are pretty confident that there is no copper mining in KR, TW, JP, or DE.

As such, we can map the Exiobase net copper (monetary) fractions to the ecoinvent mining regions:

In [21]:
NO_MINING = ["KR", "JP", "TW", "DE"]

mapping = {
    "WL": "CL",  
    "CN": "CN",
    "WM": "RoW",
    "US": "US",
    "WA": "KZ",  
    "IN": "RoW",
    "AU": "AU", 
    "ES": "RoW",
    "WF": "ZM",
    "MX": "RoW",
    "ID": "ID",
}

The `IOTable` doesn't really support modification, so if we wanted our copper changes together with our aluminium changes the best option would be to put everything together in a single notebook and write to one database. Here, we will create a new database just for copper, following the same pattern as in notebook 3.

First, we need to know how much copper to substitute with our new market mix. This isn't trivial, as the following result from `ba.print_recursive_calculation(motor, ipcc, cutoff=0.025)` makes me a bit suspicious:

    Fraction of score | Absolute score | Amount | Activity
    0001 | 9.134 |     1 | 'electric motor production, vehicle (electric powertrain)' (kilogram, 
      0.0975 | 0.8903 | 0.1349 | 'market for copper, cathode' (kilogram, GLO, None)
        0.0726 | 0.6628 | 0.08161 | 'electrorefining of copper, anode' (kilogram, GLO, None)
          0.052 | 0.4751 | 0.081 | 'market for copper, anode' (kilogram, GLO, None)

We can just do an inventory calculation to get the copper needed for one kilogram of `'market for copper, cathode'`:

In [27]:
lca = bc.LCA(
    {bd.get_activity(
        name="market for copper, cathode", 
        database="ei 3.8 cutoff", 
    ): 1}
)
lca.lci()
copper_mass_in_cathode = lca.supply_array[lca.dicts.product[copper.id]]
copper_mass_in_cathode

2.199497333229909

I guess there is a lot of waste during recycling; presumably most of this is recycled, but in cutoff system models we don't get credit for production of recyclable materials. So we then need this much copper:

In [28]:
net_copper = copper_mass_in_cathode * 0.1349

We can now just copy/paste the approach from notebook three; we have our net amount, our new market mix, etc.

In [30]:
from collections import defaultdict

cu_market = defaultdict(float)

for amount, node in copper_inputs_kr_elec_machinery[:15]:
    if node['location'] in NO_MINING:
        continue
    cu_market[bd.get_activity(
        database="ei 3.8 cutoff",
        name='copper mine operation and beneficiation, sulfide ore',
        product='copper concentrate, sulfide ore',
        location=mapping[node['location']]
    )] += amount

In [31]:
total = sum(cu_market.values())

cu_market = {key: value / total for key, value in cu_market.items()}
cu_market

{'copper mine operation and beneficiation, sulfide ore' (kilogram, CL, None): 0.3244048929180896,
 'copper mine operation and beneficiation, sulfide ore' (kilogram, CN, None): 0.24283398272025009,
 'copper mine operation and beneficiation, sulfide ore' (kilogram, RoW, None): 0.20666633925025507,
 'copper mine operation and beneficiation, sulfide ore' (kilogram, US, None): 0.09549119287409931,
 'copper mine operation and beneficiation, sulfide ore' (kilogram, KZ, None): 0.06264766014814435,
 'copper mine operation and beneficiation, sulfide ore' (kilogram, AU, None): 0.029752681525315197,
 'copper mine operation and beneficiation, sulfide ore' (kilogram, ZM, None): 0.02145962616990471,
 'copper mine operation and beneficiation, sulfide ore' (kilogram, ID, None): 0.01674362439394173}

In [32]:
if "강한 국가를 위한 영광스러운 구리" in bd.databases:
    del bd.databases["강한 국가를 위한 영광스러운 구리"]

In [33]:
kr = bd.Database("강한 국가를 위한 영광스러운 구리", backend = "iotable")
type(kr)

bw2data.backends.iotable.backend.IOTableBackend

In [34]:
kr.write({})

In [35]:
new_motor = kr.new_activity(code="em-kr", name="Electric motor", location="KR", unit="kilogram")
new_cu = kr.new_activity(code="cu-kr", name="Korea-specific copper mix", location="KR", unit="kilogram")

In [36]:
new_motor.save()
new_cu.save()

In [38]:
edges = [
    # Motor activity
    {"row": new_motor.id, "col": new_motor.id, "amount": 1},  # production exchange,
    {"row": motor.id, "col": new_motor.id, "amount": 1, "flip": True},   # Need the rest of the motor
    {"row": new_cu.id, "col": new_motor.id, "amount": net_copper, "flip": True},  # Substitute this much aluminium
] + [
    # New copper mix
    {"row": new_cu.id, "col": new_cu.id, "amount": 1},  # production exchange,
    {"row": copper.id, "col": new_cu.id, "amount": 1},   # This is the substituted aluminium mix
] + [
    {"row": node.id, "col": new_cu.id, "amount": amount, "flip": True} for node, amount in cu_market.items()
]

In [39]:
kr.write_exchanges(edges, [], ["ei 3.8 cutoff"])

Starting IO table write
Adding technosphere matrix
Adding biosphere matrix
Finalizing serialization


In [40]:
motor.new_edge(input=new_motor, amount=0, type="technosphere").save()

In [41]:
_, data_objs, _ = bd.prepare_lca_inputs({motor: 1}, ipcc)

In [42]:
lca = bc.LCA({motor.id: 1}, data_objs=data_objs)
lca.lci()
lca.lcia()
lca.score

9.134067244425337

In [43]:
lca.lcia({new_motor.id: 1})
lca.score

9.139341861641421